In [1]:
from datetime import date

param_startDate = date.today()

In [2]:
import pandas as pd

data = {
    "Task Name": ["Task 1", "Task 2", "Task 3", "Task 4"],
    "Maximum Resources": [10, 5, 8, 0],
    "Effort": [14, 6, 3, 5],
    "Execution Order": [1, 2, 2, 3]
}

df = pd.DataFrame(data)
df


,Task Name,Maximum Resources,Effort,Execution Order
0,Task 1,10,14,1
1,Task 2,5,6,2
2,Task 3,8,3,2
3,Task 4,0,5,3


### Github copilot algo

In [3]:
import pandas as pd
from datetime import timedelta, date

# Create a copy of the original dataframe
schedule = df.copy()

# Initialize the "Start Date" column
schedule['Start Date'] = schedule.apply(lambda row: param_startDate if row['Execution Order'] == 1 else None, axis=1)

schedule = schedule.sort_values('Execution Order')
schedule


# Calculate the "Finish Date" for each task
for index, row in schedule.iterrows():
    # Check if the task has dependencies
    if row['Execution Order'] > 1:
        # Find the maximum "Finish Date" of the dependent tasks
        dependencies = schedule.loc[schedule['Execution Order'] < row['Execution Order']]
        max_finish_date = dependencies['Finish Date'].max()
        
        # Calculate the "Start Date" based on the maximum "Finish Date" of dependencies
        if pd.notnull(max_finish_date):
            start_date = max_finish_date + timedelta(days=1)
            
            # Skip weekends (Saturday and Sunday)
            while start_date.weekday() >= 5:
                start_date += timedelta(days=1)
            
            schedule.at[index, 'Start Date'] = start_date
            ##print(schedule)
    
    # Calculate the "Finish Date" based on the "Start Date" and "Effort"
    start_date = schedule.at[index, 'Start Date']
    effort = row['Effort']
    finish_date = start_date + timedelta(days=effort)
    
    # Skip weekends (Saturday and Sunday)
    while finish_date.weekday() >= 5:
        finish_date += timedelta(days=1)
    
    schedule.at[index, 'Finish Date'] = finish_date

schedule


,Task Name,Maximum Resources,Effort,Execution Order,Start Date,Finish Date
0,Task 1,10,14,1,2024-05-07,2024-05-21
1,Task 2,5,6,2,2024-05-22,2024-05-28
2,Task 3,8,3,2,2024-05-22,2024-05-27
3,Task 4,0,5,3,2024-05-29,2024-06-03


### TODO: implement Prioirity based task scheduling
The goal is to:
 - Execute the tasks in execution order
 - Minimize Finish time
 - Ignore Max resources for now.

In [4]:
from datetime import datetime
def getSchedule(df, param_startDate): 
    orders = [ [] for _ in range(df["Execution Order"].max()) ]

    for row in df.to_numpy().tolist():
        exec_order = row[3]
        orders[exec_order - 1].append(row)

    data = []
    
    start_date = param_startDate
    max_finish_date = param_startDate - timedelta(days=1)
    for order in orders:
        start_date = max_finish_date + timedelta(days=1)
        while start_date.weekday() >= 5:
            start_date += timedelta(days=1)
        for row in order:
            effort = row[2]
            finish_date = start_date + timedelta(days=effort)
            while finish_date.weekday() >= 5:
                finish_date += timedelta(days=1)
            max_finish_date = max(finish_date, max_finish_date)
            row.append(start_date.strftime('%Y-%m-%d'))
            row.append(finish_date.strftime('%Y-%m-%d'))
            data.append(row)
    
    schedule = pd.DataFrame(data, columns=['Task Name', 'Maximum Resources', 'Effort', 'Execution Order', 'Start Date', 'Finish Date'])
    return schedule

schedule = getSchedule(df, datetime.strptime('2024-05-06', '%Y-%m-%d'))
schedule

,Task Name,Maximum Resources,Effort,Execution Order,Start Date,Finish Date
0,Task 1,10,14,1,2024-05-06,2024-05-20
1,Task 2,5,6,2,2024-05-21,2024-05-27
2,Task 3,8,3,2,2024-05-21,2024-05-24
3,Task 4,0,5,3,2024-05-28,2024-06-03


In [5]:
data = {
            "Task Name": ["Task 1", "Task 2", "Task 3", "Task 4"],
            "Maximum Resources": [10, 5, 8, 0],
            "Effort": [14, 6, 3, 5],
            "Execution Order": [1, 2, 2, 3],
            "Start Date": ['2024-05-06', '2024-05-21', '2024-05-21', '2024-05-28'],
            "Finish Date": ['2024-05-20', '2024-05-27', '2024-05-24', '2024-06-03']        
        }
expected = pd.DataFrame(data)
expected

,Task Name,Maximum Resources,Effort,Execution Order,Start Date,Finish Date
0,Task 1,10,14,1,2024-05-06,2024-05-20
1,Task 2,5,6,2,2024-05-21,2024-05-27
2,Task 3,8,3,2,2024-05-21,2024-05-24
3,Task 4,0,5,3,2024-05-28,2024-06-03


In [6]:
pd.testing.assert_frame_equal(schedule,expected,check_names=False, check_dtype=False)